First we load iris and split it into training and test sets.

In [1]:
from sklearn.datasets import load_iris
iris = load_iris()
from sklearn.model_selection import train_test_split
iris_X_train, iris_X_test, iris_y_train, iris_y_test = train_test_split(iris['data'], iris['target'], random_state=0)

What are the sizes of the training and test sets?

In [3]:
iris['data']

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1],
       [5.4, 3.7, 1.5, 0.2],
       [4.8, 3.4, 1.6, 0.2],
       [4.8, 3. , 1.4, 0.1],
       [4.3, 3. , 1.1, 0.1],
       [5.8, 4. , 1.2, 0.2],
       [5.7, 4.4, 1.5, 0.4],
       [5.4, 3.9, 1.3, 0.4],
       [5.1, 3.5, 1.4, 0.3],
       [5.7, 3.8, 1.7, 0.3],
       [5.1, 3.8, 1.5, 0.3],
       [5.4, 3.4, 1.7, 0.2],
       [5.1, 3.7, 1.5, 0.4],
       [4.6, 3.6, 1. , 0.2],
       [5.1, 3.3, 1.7, 0.5],
       [4.8, 3.4, 1.9, 0.2],
       [5. , 3. , 1.6, 0.2],
       [5. , 3.4, 1.6, 0.4],
       [5.2, 3.5, 1.5, 0.2],
       [5.2, 3.4, 1.4, 0.2],
       [4.7, 3.2, 1.6, 0.2],
       [4.8, 3.1, 1.6, 0.2],
       [5.4, 3.4, 1.5, 0.4],
       [5.2, 4.1, 1.5, 0.1],
       [5.5, 4.2, 1.4, 0.2],
       [4.9, 3

In [2]:
iris_X_train.shape

(112, 4)

In [3]:
iris_X_test.shape

(38, 4)

Let us now load the ionosphere dataset:

In [4]:
import numpy as np
is_data = np.genfromtxt("ionosphere.txt", delimiter=",", usecols=np.arange(34))
is_target = np.genfromtxt("ionosphere.txt", delimiter=",", usecols=34, dtype='int')

Splitting it into training and test sets:

In [5]:
is_X_train, is_X_test, is_y_train, is_y_test = train_test_split(is_data, is_target, random_state=0)

What are the sizes of the training and test sets?

In [6]:
is_X_train.shape

(263, 34)

In [7]:
is_X_test.shape

(88, 34)

## Nearest Neighbour

In this section we implement the basic Nearest Neighbour.

### Nearest Neighbour for iris

First let us write a simple function for computing Euclidean distance and test it.

In [8]:
def dist(x1,x2):
  return np.linalg.norm(x1-x2)
dist(np.array([0,0]),np.array([1,1]))

1.4142135623730951

An alternative to using the function np.linalg.norm, computing the Euclidean norm of a vector, is the function you wrote for Exercise 6 in Section 7 of Lab Worksheet 3.  (That function computes the squared Euclidean norm, but this will not affect the output of the Nearest Neighbour algorithm or the conformal predictor based on it.)

The next function finds the nearest neighbour to x in X. We also have a very primitive test of the code.

In [9]:
import math
def nearest(x,X):
  current_record = math.inf
  for n in range(X.shape[0]):
    current_dist = dist(x,X[n])
    if current_dist < current_record:
      current_record = current_dist
      current_record_holder = n
  return current_record, current_record_holder
nearest(np.array([1,1,0]),np.zeros((3,3)))

(1.4142135623730951, 0)

Now we can go over the test set applying the function "nearest" to each test sample.  Along the way we compute the number ot errors and print all errors.

In [10]:
import time
start = time.time()
n_iris_test = iris_X_test.shape[0]
n_errors = 0
prediction = np.zeros(n_iris_test,dtype=int)
for n in range(n_iris_test):
  prediction[n] = iris_y_train[nearest(iris_X_test[n],iris_X_train)[1]]
  if prediction[n] != iris_y_test[n]:
    n_errors = n_errors + 1
    print("Error:",n)
print("Number of errors:",n_errors)
print("Error rate:",n_errors / n_iris_test)
print(time.time() - start,"seconds")

Error: 37
Number of errors: 1
Error rate: 0.02631578947368421
0.03127408027648926 seconds


And let's see our predictions and the actual labels.

In [11]:
prediction

array([2, 1, 0, 2, 0, 2, 0, 1, 1, 1, 2, 1, 1, 1, 1, 0, 1, 1, 0, 0, 2, 1,
       0, 0, 2, 0, 0, 1, 1, 0, 2, 1, 0, 2, 2, 1, 0, 2])

In [12]:
iris_y_test

array([2, 1, 0, 2, 0, 2, 0, 1, 1, 1, 2, 1, 1, 1, 1, 0, 1, 1, 0, 0, 2, 1,
       0, 0, 2, 0, 0, 1, 1, 0, 2, 1, 0, 2, 2, 1, 0, 1])

### Nearest Neighbour for ionosphere

The program for ionospehere is very similar, but now we do not bother remembering the predictions.

In [13]:
start = time.time()
n_is_test = is_X_test.shape[0]
n_errors = 0
for n in range(n_is_test):
  prediction = is_y_train[nearest(is_X_test[n],is_X_train)[1]]
  if prediction != is_y_test[n]:
    print("Error:",n)
    n_errors = n_errors + 1
print("Number of errors:",n_errors)
print("Error rate:",n_errors / n_is_test)
print(time.time() - start,"seconds")

Error: 2
Error: 6
Error: 9
Error: 34
Error: 42
Error: 47
Error: 50
Error: 54
Error: 55
Error: 69
Error: 75
Error: 83
Error: 85
Number of errors: 13
Error rate: 0.14772727272727273
0.16929244995117188 seconds


### Summary of the results

The results for random state 0 are: 97% accuracy on iris and 85% accuracy for ionosphere.

## Conformal predictors

In this section we will implement conformal predictors in the conceptually cleanest way following their definition.  They are, however, computationally inefficient and take a few minutes to run on my laptop.  If you would like to experiment with conformal predictors, you will be better off moving to the next section.

### Conformal predictor for iris

First we have a function computing the confomity score of the kth element of a bag (X,y) (where X is the data and y is the labels).

In [14]:
def conformity(X,y,k):   # test object: the kth row
  current_record_own = current_record_other = math.inf
  for n in range(X.shape[0]):
    current_dist = dist(X[k],X[n])
    if (current_dist < current_record_own) and (y[n]==y[k]) and (n!=k):
      current_record_own = current_dist
    if (current_dist < current_record_other) and (y[n]!=y[k]) and (n!=k):
      current_record_other = current_dist
  if current_record_own == 0:
    if (current_record_other == 0):
      print("Division 0/0")    # sanity check
      return 0                 # see below
    return math.inf
  else:
    return current_record_other / current_record_own

In [15]:
n_iris_train = iris_X_train.shape[0]
n_iris_test = iris_X_test.shape[0]
n_errors = 0
sum_p_values = 0.0
prediction = np.zeros(n_iris_test,dtype=int)
score = np.zeros(n_iris_train+1)  # the conformity scores
p = np.zeros(3)    # the p-values
for n in range(n_iris_test):
  augmented_X = np.row_stack((iris_X_train,iris_X_test[n]))
  for l in range(3):   # postulated label
    augmented_y = np.append(iris_y_train,l)
    for m in range(n_iris_train+1):
      score[m] = conformity(augmented_X,augmented_y,m)
    p[l] = np.mean(score<=score[n_iris_train])  # the p-value of l
    # the formula in the lectures: p[l] = sum(score<=score[n_iris_train]) / (n_iris_train+1)
  prediction[n] = np.argmax(p)   # the smallest argmax
  if prediction[n] != iris_y_test[n]:
    n_errors = n_errors + 1
    print("Error: ",n)
  sum_p_values = sum_p_values + p[0] + p[1] + p[2] - p[iris_y_test[n]]
print("Number of errors: ", n_errors)
print("Mean false p-value: ", sum_p_values/(2*n_iris_test))

Error:  37
Number of errors:  1
Mean false p-value:  0.01082906380996741


I am getting four instances of "Division 0/0" for random state 0.  Is it really true that there are identical samples that belong to different classes in the iris dataset?

In [16]:
n_iris = iris['data'].shape[0]
for i in range(n_iris-1):
  for j in range(i+1,n_iris):
    current_dist = dist(iris['data'][i],iris['data'][j])
    if (current_dist == 0): # and ():
      print("Samples ", i, " and ",j," are identical, with labels ",iris['target'][i], " and ", iris['target'][j])

Samples  101  and  142  are identical, with labels  2  and  2


There are no identical samples with different labels!  What do you think went wrong?  And why do you think I defined 0/0 to be 0 in the function "conformity"?  (By the way, it's a standard convention.)

### Conformal predictor for ionosphere

The following program is very similar, but there is an important source of complications: the labels are {-1,1} rather than {0,1}.  We still code the labels by {0,1}.

In [17]:
n_is_train = is_X_train.shape[0]
n_is_test = is_X_test.shape[0]
n_errors = 0
sum_p_values = 0.0
prediction = np.zeros(n_is_test,dtype=int)
score = np.zeros(n_is_train+1)  # the conformity scores
p = np.zeros(2)    # the p-values
for n in range(n_is_test):
  augmented_X = np.vstack((is_X_train,is_X_test[n]))
  for l in range(2):   # code of the postulated label, 0 standing for -1 and 1 for 1
    augmented_y = np.append(is_y_train,2*l-1)  # 2*l-1 is the postulated label
    for m in range(n_is_train+1):
      score[m] = conformity(augmented_X,augmented_y,m)
    p[l] = np.mean(score<=score[n_is_train])  # the p-value of l
    # p[l] = sum(score<=score[n_is_train]) / (n_is_train+1)
  prediction[n] = 2*np.argmax(p)-1   # the smallest argmax, and the code is transformed into the label
  if prediction[n] != is_y_test[n]:
    n_errors = n_errors + 1
    print("Error: ",n)
  sum_p_values = sum_p_values + p[0] + p[1] - p[(is_y_test[n]+1)//2]  # in p[...]: turning label into code
print("Number of errors: ", n_errors)
print("Error rate: ",n_errors / n_is_test)
print("Mean false p-value: ", sum_p_values/n_is_test)

Error:  2
Error:  6
Error:  9
Error:  34
Error:  42
Error:  47
Error:  50
Error:  54
Error:  55
Error:  69
Error:  75
Error:  83
Error:  85
Number of errors:  13
Error rate:  0.14772727272727273
Mean false p-value:  0.06099345730027552


### Brief discussion of results

It is interesting that the simple Nearest Neighbour and conformalized Nearest Neighbour make the same numbers of errors and, moreover, they make errors on the same test samples.

## More efficient conformal predictors (with preprocessing the training set)

The conformal predictors of the previous section are extremely inefficient; we kept performing the same calculations over and over again.  We make them more efficient by preprocessing the training set.  Exercise: think of further ways of improving efficiency.  Ideally, the number of calls to dist should be n(n-1)/2+kn, where n is the size of the training set and k is the size of the test set.

### A more efficient conformal predictor for iris

First we preprocess the training set by computing the distance of each training sample to its nearest neghbour in the training set of the same and other class.

In [18]:
# The following two commands are only needed if you skipped the previous section.
# (But they are harmless.)
n_iris_train = iris_X_train.shape[0]
n_iris_test = iris_X_test.shape[0]
dist_own = math.inf * np.ones(n_iris_train)  # shortest distances to the own class, initialized to infinity
dist_other = math.inf * np.ones(n_iris_train)  # shortest distances to the other class, initialized to infinity
for i in range(n_iris_train-1):
  for j in range(i+1,n_iris_train):
    current_dist = dist(iris_X_train[i],iris_X_train[j])
    if iris_y_train[i]==iris_y_train[j]:
      if (current_dist < dist_own[i]):
        dist_own[i] = current_dist
      if (current_dist < dist_own[j]):
        dist_own[j] = current_dist
    else:
      if (current_dist < dist_other[i]):
        dist_other[i] = current_dist
      if (current_dist < dist_other[j]):
        dist_other[j] = current_dist

Now we are ready to process the test set.

In [19]:
n_errors = 0
sum_p_values = 0.0
prediction = np.zeros(n_iris_test,dtype=int)
score = np.zeros(n_iris_train+1)  # the conformity scores
p = np.zeros(3)    # the p-values

for j in range(n_iris_test):
  for l in range(3):   # postulated label
    aug_dist_own = np.append(dist_own,math.inf)
    aug_dist_other = np.append(dist_other,math.inf)
    for i in range(n_iris_train):
      current_dist = dist(iris_X_train[i],iris_X_test[j])
      if iris_y_train[i]==l:
        if (current_dist < aug_dist_own[i]):
          aug_dist_own[i] = current_dist
        if (current_dist < aug_dist_own[n_iris_train]):
          aug_dist_own[n_iris_train] = current_dist
      else:
        if (current_dist < aug_dist_other[i]):
          aug_dist_other[i] = current_dist
        if (current_dist < aug_dist_other[n_iris_train]):
          aug_dist_other[n_iris_train] = current_dist
    # the following for loop is the careful version of score = aug_dist_other / aug_distance_own
    for i in range(n_iris_train+1):
      if aug_dist_own[i] == 0:
        score[i] = math.inf
        if (aug_dist_other[i] == 0):
          print("Division 0/0")    # sanity check
          score[i] = 0
      else:
        score[i] = aug_dist_other[i] / aug_dist_own[i]
    p[l] = np.mean(score<=score[n_iris_train])  # the p-value of l
    # p[l] = sum(score<=score[n_iris_train]) / (n_iris_train+1)  # as in Chapter 3
  prediction[j] = np.argmax(p)   # the smallest argmax
  if prediction[j] != iris_y_test[j]:
    n_errors = n_errors + 1
    print("Error: ",j)
  sum_p_values = sum_p_values + p[0] + p[1] + p[2] - p[iris_y_test[j]]
print("Number of errors: ", n_errors)
print("Mean false p-value: ", sum_p_values/(2*n_iris_test))

Error:  37
Number of errors:  1
Mean false p-value:  0.01082906380996741


### A more efficient conformal predictor for ionosphere

The program for ionosphere is very similar, but with the same complication as in the previous section.  First we preprocess the dataset.

In [20]:
# The following two commands are only needed if you skipped the previous section.
n_is_train = is_X_train.shape[0]
n_is_test = is_X_test.shape[0]
dist_own = math.inf * np.ones(n_is_train)  # shortest distances to the own class, initialized to infinity
dist_other = math.inf * np.ones(n_is_train)  # shortest distances to the own class, initialized to infinity
for i in range(n_is_train-1):
  for j in range(i+1,n_is_train):
    current_dist = dist(is_X_train[i],is_X_train[j])
    if is_y_train[i]==is_y_train[j]:
      if (current_dist < dist_own[i]):
        dist_own[i] = current_dist
      if (current_dist < dist_own[j]):
        dist_own[j] = current_dist
    else:
      if (current_dist < dist_other[i]):
        dist_other[i] = current_dist
      if (current_dist < dist_other[j]):
        dist_other[j] = current_dist

Now we can process the test set more efficiently.

In [21]:
n_errors = 0
sum_p_values = 0.0
prediction = np.zeros(n_is_test,dtype=int)
score = np.zeros(n_is_train+1)  # the conformity scores
p = np.zeros(2)    # the p-values

for j in range(n_is_test):
  for l in range(2):   # postulated label code
    aug_dist_own = np.append(dist_own,math.inf)
    aug_dist_other = np.append(dist_other,math.inf)
    for i in range(n_is_train):
      current_dist = dist(is_X_train[i],is_X_test[j])
      if is_y_train[i]==2*l-1:
        if (current_dist < aug_dist_own[i]):
          aug_dist_own[i] = current_dist
        if (current_dist < aug_dist_own[n_is_train]):
          aug_dist_own[n_is_train] = current_dist
      else:
        if (current_dist < aug_dist_other[i]):
          aug_dist_other[i] = current_dist
        if (current_dist < aug_dist_other[n_is_train]):
          aug_dist_other[n_is_train] = current_dist
    # the following for loop is the careful version of score = aug_dist_other / aug_distance_own
    for i in range(n_is_train+1):
      if aug_dist_own[i] == 0:
        score[i] = math.inf
        if (aug_dist_other[i] == 0):
          print("Division 0/0")    # sanity check
          score[i] = 0
      else:
        score[i] = aug_dist_other[i] / aug_dist_own[i]
    p[l] = np.mean(score<=score[n_is_train])  # the p-value of l
    # p[l] = sum(score<=score[n_is_train]) / (n_is_train+1)
  prediction[j] = 2*np.argmax(p)-1   # the smallest argmax, turned into a label
  if prediction[j] != is_y_test[j]:
    n_errors = n_errors + 1
    print("Error: ",j)
  sum_p_values = sum_p_values + p[0] + p[1] - p[(is_y_test[j]+1)//2]
print("Number of errors: ", n_errors)
print("Mean false p-value: ", sum_p_values/n_is_test)

Error:  2
Error:  6
Error:  9
Error:  34
Error:  42
Error:  47
Error:  50
Error:  54
Error:  55
Error:  69
Error:  75
Error:  83
Error:  85
Number of errors:  13
Mean false p-value:  0.06099345730027552


### Remark about the results

Of course, the results we are getting in this section are identical to those in the previous section (we have just removed superfluous computations).